<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Movie-recommendation" data-toc-modified-id="Movie-recommendation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Movie recommendation</a></span><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Evaluation-Protocol" data-toc-modified-id="Evaluation-Protocol-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Evaluation Protocol</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#ALS" data-toc-modified-id="ALS-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback" target="_blank">ALS</a></a></span></li><li><span><a href="#Ваша-формулировка" data-toc-modified-id="Ваша-формулировка-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Ваша формулировка</a></span></li></ul></li><li><span><a href="#Evaluation-Results" data-toc-modified-id="Evaluation-Results-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Evaluation Results</a></span></li></ul></li></ul></div>

# Movie recommendation

Ваша задача - рекомендация фильмов для пользователей


In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "40g") \
    .config("spark.driver.memory", "40g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

## Dataset 

`MovieLens-25M`

In [2]:
!ls /workspace/ml_bd/ml-25m

genome-scores.csv  links.csv   rating_graph.edgelist  README.txt
genome-tags.csv    movies.csv  ratings.csv	      tags.csv


In [2]:
DATA_PATH = '/workspace/ml_bd/ml-25m'

RATINGS_PATH = os.path.join(DATA_PATH, 'ratings.csv')
MOVIES_PATH = os.path.join(DATA_PATH, 'movies.csv')
TAGS_PATH = os.path.join(DATA_PATH, 'tags.csv')

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *


ratings_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + RATINGS_PATH)

In [5]:
movies_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + MOVIES_PATH)

In [6]:
ratings_df.take(1)

[Row(userId=1, movieId=296, rating=5.0, timestamp=1147880044)]

In [7]:
movies_df.take(1)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy')]

## Evaluation Protocol

Так как мы хотим оценивать качество разных алгоритмов рекомендаций, то в первую очередь нам нужно определить
* Как разбить данные на `Train`/`Validation`/`Test`;
* Какие метрики использовать для оценки качества.

In [4]:
from metrics import get_ate
from processing import split_by_col, partitioned_split_by_cols

### Protocol 1 --- split by timestamp

In [6]:
train_df, test_df = split_by_col(ratings_df, 'timestamp', [0.85, 0.15])

In [7]:
new_test_users_num = test_df.select('userId').distinct().subtract(train_df.select('userId').distinct()).count()
print("Number of users in test but not in train:", new_test_users_num)

Number of users in test but not in train: 17993


In [8]:
print("Number of users in train:", train_df.select('userId').distinct().count())
print("Number of users in test:", test_df.select('userId').distinct().count())

Number of users in train: 144548
Number of users in test: 24525


In [9]:
print("Number in train:", train_df.count())
print("Number in test:", test_df.count())

Number in train: 21250080
Number in test: 3750014


In [7]:
good_users = test_df.select('userId').distinct().intersect(train_df.select('userId').distinct()).collect()
good_users = [user['userId'] for user in good_users]
test_df = test_df\
    .filter(F.col('userId').isin(good_users))


### Protocol 2 --- split each user by timestamp and unite

In [5]:
train_df, val_df, test_df = partitioned_split_by_cols(ratings_df, 'userId', 'timestamp', [0.8, 0.1, 0.1])

In [11]:
new_test_users_num = test_df.select('userId').distinct().subtract(train_df.select('userId').distinct()).count()
new_val_users_num = val_df.select('userId').distinct().subtract(train_df.select('userId').distinct()).count()
print("Number of users in val but not in train:", new_val_users_num)
print("Number of users in test but not in train:", new_test_users_num)

Number of users in val but not in train: 0
Number of users in test but not in train: 0


In [6]:
print("Number of users in train:", train_df.select('userId').distinct().count())
print("Number of users in val:", val_df.select('userId').distinct().count())
print("Number of users in test:", test_df.select('userId').distinct().count())

Number of users in train: 162541
Number of users in val: 158503
Number of users in test: 157054


In [13]:
print("Number in train:", train_df.count())
print("Number in val:", val_df.count())
print("Number in test:", test_df.count())

Number in train: 20017461
Number in val: 2477991
Number in test: 2367777


### Protocols consensus

В первом способе получилось слишком много новых пользователей в тесте и валидейте, так что использовать в так виде или оставлять только "общих" пользователей кажется бессмысленным. Во втором способе этой проблемы нет. Теоретическая проблема: заглядывание в будущее. Но мы по сути заглядываем в будущее для оценок фильмов. А фильмы это фиксированный объект, со временем не меняется. Так что кажется, что ничего страшного в этом нет.

Далее используется разделение датасета вторым способом.

Считаем, что стоит рекомендовать посмотреть, если рейтинг поставленный >=3.

In [7]:
test_df = test_df.filter(F.col('rating') >= 3.)
val_df = val_df.filter(F.col('rating') >= 3.)

### Protocols consensus

При рекомендации фильмов обычно число k в top-k не очень большое, так что, кажется, порядок не так важен. Интереса ради используется одна метрика порядка (MAP). Но основными метриками будут Precision@k, Recall@k; [1, 5, 10]. HR@k не используется, так как не очень интересно, сколько раз удалось пользователю посоветовать ХОТЬ ОДИН хороший фильм. Хочется скорее узнать среднее число удачных советов.

In [8]:
def recall_at_k(recommended_df, true_df, k=10):
    def limiter(lst):
        return lst[:k]

    limiter_udf = F.udf(limiter, ArrayType(IntegerType()))

    users_num = true_df.select('userId').distinct().count()
    result = recommended_df\
        .withColumn('recommendations', limiter_udf('recommendations'))\
        .join(true_df, 'userId')\
        .withColumn('common', F.size(F.array_intersect('recommendations', 'golds')) / F.size(F.col('golds')))\
        .select('common')\
        .groupBy().agg(F.sum('common').alias('sum')).take(1)[0]['sum']
    return result / users_num

In [9]:
from pyspark.sql import Window
from pyspark.mllib.evaluation import RankingMetrics

def evaluate_movies(recommend_df, gold_df, tops=[1, 5, 10]):
    metrics = {}
    golds = gold_df\
        .groupBy('userId')\
        .agg(F.collect_set('movieId').alias('golds'))
    rdd = golds\
        .join(recommend_df, 'userId') \
        .select('recommendations', 'golds') \
        .rdd
    ranking_metrics = RankingMetrics(rdd)
    
    for i in tops:
        metrics['precision@' + str(i)] = ranking_metrics.precisionAt(i)
        metrics['recall@' + str(i)] = recall_at_k(recommend_df, golds, i)
    metrics['MAP'] = ranking_metrics.meanAveragePrecision
    return metrics

## Models

Теперь мы можем перейти к формулировке задачи в терминах машинного обучения.

Одна из формулировок, к которой мы сведем нашу задачу - **Matrix Completetion**. Данную задачу будем решать с помощью `ALS`

### [ALS](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback)

In [11]:
baseline_als_space = {
    'rank': 10,
    'maxIter': 10, 
    'regParam': 0.1, 
    'numUserBlocks': 10, 
    'numItemBlocks': 10, 
    'implicitPrefs': False, 
    'alpha': 1.0, 
    'userCol': 'userId', 
    'itemCol': 'movieId', 
    'seed': 42, 
    'ratingCol': 'rating', 
    'nonnegative': False, 
    'checkpointInterval': 10, 
    'intermediateStorageLevel': 'MEMORY_AND_DISK', 
    'finalStorageLevel': 'MEMORY_AND_DISK', 
    'coldStartStrategy': 'nan'
}

In [12]:
from pyspark.ml.recommendation import ALS

def train_als(space, df):
    model = ALS(**space)
    model = model.fit(df)
    watched = train_df\
        .groupby('userId')\
        .agg(F.collect_set('movieId').alias('watched'))\
        .withColumnRenamed('userId', 'user')
    return model, watched

In [13]:
from pyspark.ml.recommendation import ALS
base_als = ALS(**baseline_als_space)
base_als_model, watched_df = train_als(baseline_als_space, train_df)
watched_df = watched_df.cache()
watched_df.take(1)

[Row(user=148, watched=[3681, 1199, 589, 3000, 92259, 1222, 1193, 1172, 1266, 44191, 912, 134853, 527, 19, 2959, 608, 104879, 77455, 4993, 2176, 50, 68954, 296, 1089, 1233, 81834, 908, 5060, 858, 116897, 2324, 1206, 57669, 112556, 48516, 750, 1208, 111, 109487, 904, 1252, 99114, 7132, 2951, 919, 78499, 6016, 1196, 115617, 79132, 1198, 923, 1250, 54286, 1221, 122886, 32, 7153, 1244, 1267, 1136, 48394, 1217, 1086, 2502, 2858, 117533, 106100, 2329, 58559, 1292, 5952, 88810, 152059, 6787, 3462, 3949, 1213, 4886, 909, 2019, 134130, 1207, 953, 924, 8228, 1209, 955, 541, 1080, 2067, 1203, 318, 593, 1276, 899, 27773, 60069, 260, 1197, 63082, 922, 1270, 2571, 110])]


Покажите для выбранных вами фильмов топ-20 наиболее похожих фильмов

In [19]:
base_als_vectors = base_als_model.itemFactors
base_als_vectors.take(2)

[Row(id=10, features=[-0.3731934428215027, 0.6107021570205688, 0.31835755705833435, 0.347337543964386, 0.7369728088378906, -0.7249520421028137, -0.4799730181694031, 0.1913396418094635, 0.8426313996315002, -0.6113075017929077]),
 Row(id=20, features=[-0.46242234110832214, 0.47487494349479675, -0.004481089301407337, 0.02912776544690132, 0.4901317358016968, -0.896612286567688, -0.26179176568984985, 0.3643686771392822, 0.8532597422599792, -0.1589564085006714])]

In [21]:
def recommendations_to_titles(data):
    tmp = movies_df.select('movieId', 'title')\
        .join(data, movies_df['movieId'] == data['baseId'])\
        .drop('movieId')\
        .withColumnRenamed('title', 'baseTitle')
    return movies_df.select('movieId', 'title')\
        .join(tmp, movies_df['movieId'] == tmp['recommendationId'])\
        .drop('movieId')\
        .withColumnRenamed('title', 'recommendationTitle')\
        .sort(F.col('score').desc())

In [22]:
from pyspark.sql.functions import udf, array
from scipy.spatial.distance import cosine


def cosine_sim(array):
        return float(1 - cosine(array[0], array[1]))


def find_similar_movie(vectors_df, movieIds, topK):
    base_df = vectors_df\
        .filter(F.col('id').isin(movieIds))\
        .withColumnRenamed('id', 'baseId')\
        .withColumnRenamed('features', 'baseFeatures')
    
    dot_udf = udf(cosine_sim, FloatType())

    
    result = vectors_df\
        .join(base_df, base_df['baseId'] != vectors_df['id'])\
        .withColumn('score', dot_udf(array('features', 'baseFeatures')))\
        .sort(F.col('score').desc())\
        .limit(topK)\
        .drop('features', 'baseFeatures')\
        .withColumnRenamed('id', 'recommendationId')
    return result

In [24]:
movies_df.filter(F.col('movieId')==260).take(1)

[Row(movieId=260, title='Star Wars: Episode IV - A New Hope (1977)', genres='Action|Adventure|Sci-Fi')]

In [23]:
test_movies = recommendations_to_titles(find_similar_movie(base_als_vectors, [260], 20))
test_movies.take(10)

[Row(recommendationTitle='Star Wars: Episode V - The Empire Strikes Back (1980)', baseTitle='Star Wars: Episode IV - A New Hope (1977)', recommendationId=1196, baseId=260, score=0.9970183372497559),
 Row(recommendationTitle='Star Wars: Episode VI - Return of the Jedi (1983)', baseTitle='Star Wars: Episode IV - A New Hope (1977)', recommendationId=1210, baseId=260, score=0.9899550676345825),
 Row(recommendationTitle='Floating Skyscrapers (2014)', baseTitle='Star Wars: Episode IV - A New Hope (1977)', recommendationId=129841, baseId=260, score=0.9857679009437561),
 Row(recommendationTitle='Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', baseTitle='Star Wars: Episode IV - A New Hope (1977)', recommendationId=1198, baseId=260, score=0.9824265241622925),
 Row(recommendationTitle='Star Trek II: The Wrath of Khan (1982)', baseTitle='Star Wars: Episode IV - A New Hope (1977)', recommendationId=1374, baseId=260, score=0.9775526523590088),
 Row(recommendationTitl


Предсказываем по 10 фильмов для каждого юзера из тестового датасета.

Так как нет функционала для указания айдишников, которые не надо выдавать в результате (фильмы, которые уже юзер посмотрел выдавать бессмысленно), то предсказываю по 50, а затем фильтрую просмотренные и сокращаю до 10.

In [14]:
def predict_movies(model, predict_df, watched_df, top=10):
    def extractor(lst):
        return [pr['movieId'] for pr in lst if pr['rating']>=3.]
    
    filter_udf = F.udf(extractor, ArrayType(IntegerType()))
    
    filtered_df = model\
        .recommendForUserSubset(predict_df.select('userId').distinct(), 50)\
        .withColumn('recommendations', filter_udf('recommendations'))\
        .cache()
    
    def limiter(lst):
        return lst[:top]
    
    def none_replacer(lst):
        if lst:
            return lst
        else:
            return []
    
    limiter_udf = F.udf(limiter, ArrayType(IntegerType()))
    replacer_udf = F.udf(none_replacer, ArrayType(IntegerType()))
    
    result = filtered_df\
        .join(watched_df, watched_df['user'] == filtered_df['userId'], 'left')\
        .withColumn('watched', replacer_udf('watched'))\
        .withColumn('recommendations', F.array_except(F.col('recommendations'), F.col('watched')))\
        .select('userId', 'recommendations')\
        .withColumn('recommendations', limiter_udf('recommendations'))
    
    return result
    

In [15]:
base_als_result = predict_movies(base_als_model, val_df, watched_df).cache()
base_als_result.take(1)

[Row(userId=148, recommendations=[200930, 175391, 183947, 183845, 151989, 138224, 156414, 200872, 200936, 104119])]

In [65]:
#Проверка, что не рекомендует фильмы из списка запрещенных. Все метрики должны быть по нулям.
base_als_metrics = evaluate_movies(base_als_result, train_df)
base_als_metrics

{'MAP': 0.0,
 'precision@1': 0.0,
 'precision@10': 0.0,
 'precision@5': 0.0,
 'recall@1': 0.0,
 'recall@10': 0.0,
 'recall@5': 0.0}

In [16]:
base_als_metrics = evaluate_movies(base_als_result, val_df)
base_als_metrics

{'MAP': 4.841065525501631e-07,
 'precision@1': 6.37950392977443e-06,
 'precision@10': 7.017454322751866e-06,
 'precision@5': 6.3795039297744226e-06,
 'recall@1': 2.809116657760643e-09,
 'recall@10': 2.4525548413053207e-06,
 'recall@5': 1.1797960390592066e-06}

In [12]:
all_metrics = {}
all_metrics['base ALS'] = base_als_metrics

### Ваша формулировка

На лекции было еще несколько ML формулировок задачи рекомендаций. Выберете одну из них и реализуйте метод.

#### Node2vec, который не работает

In [ ]:
GRAPH_PATH = os.path.join(DATA_PATH, 'rating_graph.edgelist')
NODE2VEC_EMB_PATH = '/workspace/node2vec/result/ratings.emd'


In [11]:
import networkx as nx

graph =nx.Graph()
for line in train_df.collect():
    graph.add_edge('u' + str(line['userId']), 'm' + str(line['movieId']), weight=line['rating'])

In [ ]:
import networkx as nx
from node2vec import Node2Vec

node2vec = Node2Vec(graph, dimensions=10, walk_length=4, num_walks=1, workers=1)
model = node2vec.fit(window=5, min_count=1, batch_words=1)

Основывалась на [этой статье](file:///home/katyakos/Downloads/3298689.3347041.pdf). Согласно табличке с результатами для рекомендации музыки, первые места занимают две нейронки, которые я не очень хотела обучать. А дальше идут простые формульные алгоритмы AR и SR из [этой статьи](https://arxiv.org/pdf/1803.09587.pdf).

#### Association Rules (AR)
Session s --- chronologically ordered tuple of item click events s = (s1,s2,s3, . . . ,sm)

S_p --- set of all past sessions

User’s current session s with s_|s| being the last item interaction in s

1(a,b) is 1 in case a and b refer to the same item and 0 otherwise

score_{AR}(i,s) = (sum_{p in S_p} sum_{x=1..|p|} sum_{y=1..|p|} \[ 1(s_|s|,p_x) * 1(i,p_y) \]) / Z

То есть берет последний фильм из текущей сессии и смотрит, сколько раз рассматриваемый для рекомендации фильм был с ним в одной из старых сессий. Считаем, что у одного юзера в трейне одна сплошная сессия, а прошлые сессии --- коллекция всех юзеров.

In [22]:
from pyspark.sql.window import Window   
import pyspark.ml

class AssociationRulesModel(pyspark.ml.Model):
    
    def __init__(self, scores_df, last_movie_df, watched_movies_df, most_popular_item,
                 userCol):
        self.scores_df = scores_df.cache()
        self.last_movie_df = last_movie_df.cache()
        self.watched_movies_df = watched_movies_df.cache()
        self.most_popular_item = most_popular_item
        self.userCol = userCol
        
           
    def predict(self, df, top=10):
        last_item_df = df.select(self.userCol).distinct()\
            .join(self.last_movie_df, self.userCol, 'left')\
            .fillna(self.most_popular_item)\
            .select(self.userCol, 'last_item')
                
        all_df = last_item_df\
            .join(self.scores_df, last_item_df['last_item'] == self.scores_df['key_item'])
        
        def none_replacer(lst):
            if lst:
                return lst
            else:
                return []
            
        def watch_filter(it, arr):
            return it not in arr

        replacer_udf = F.udf(none_replacer, ArrayType(IntegerType()))
        filter_udf = F.udf(watch_filter, BooleanType())
        window = Window.partitionBy(self.userCol).orderBy(F.col('score').desc())
        
        recommends_df = all_df\
            .join(self.watched_movies_df, self.userCol, 'left')\
            .withColumn('watched', replacer_udf('watched'))\
            .filter(filter_udf('val_item', 'watched'))\
            .withColumn('rank', F.rank().over(window))\
            .filter(F.col('rank') <= top)\
            .select(self.userCol, 'val_item', 'score')\
            .withColumn('recommendations', F.collect_list('val_item').over(window))\
            .select(self.userCol, 'recommendations')\
            .distinct()
        
        return recommends_df


class AssociationRules(pyspark.ml.Estimator):
    
    def __init__(self, userCol, itemCol, orderCol):
        self.userCol = userCol
        self.itemCol = itemCol
        self.orderCol = orderCol
        
    def fit(self, df) -> AssociationRulesModel:        
        window_coocs = Window.partitionBy('key_item', 'val_item')
        window_norm = Window.partitionBy('key_item')
        
        #Эта версия получения скоров долгая, но зато по памяти нормальная
        '''schema = StructType([StructField('key_item',IntegerType(), True),
                             StructField('val_item', IntegerType(), True),
                             StructField('score', FloatType(), True)])
        scores_df = sqlContext.createDataFrame(sc.emptyRDD(), schema)
        item_list = [r['movieId'] for r in df.select(self.itemCol).distinct().collect()]
        for keyId in item_list:
            users_list = [r['userId'] for r in
                          df.filter(F.col(self.itemCol) == keyId).select(self.userCol).distinct().collect()]
            tmp = df\
                .filter(F.col(self.userCol).isin(users_list))\
                .select(self.itemCol)\
                .filter(F.col(self.itemCol) != keyId)\
                .withColumnRenamed(self.itemCol, 'val_item')
            number = tmp.count()
            tmp = tmp\
                .groupBy('val_item')\
                .agg(F.count('val_item').alias('score'))\
                .withColumn('score', F.col('score') / F.lit(number))\
                .withColumn('key_item', F.lit(keyId))
            scores_df = scores_df.union(tmp)  '''        
        
        #Эта версия по памяти вообще не влезет, но быстрее работать будет
        scores_df = df.withColumnRenamed(self.itemCol, 'key_item')\
            .join(df, self.userCol)\
            .withColumnRenamed(self.itemCol, 'val_item')\
            .filter(F.col('key_item') != F.col('val_item'))\
            .select('key_item', 'val_item')\
            .withColumn('coocs', F.count('val_item').over(window_coocs))\
            .withColumn('key_counts', F.count('val_item').over(window_norm))\
            .withColumn('score', F.col('coocs') / F.col('key_counts'))\
            .select('key_item', 'val_item', 'score')
        
        window_user = Window.partitionBy(self.userCol)
        
        last_movie_df = df\
            .withColumn('maxTime', F.max(self.orderCol).over(window_user))\
            .filter(F.col(self.orderCol) == F.col('maxTime'))\
            .withColumn('maxItem', F.max(self.itemCol).over(window_user))\
            .filter(F.col(self.itemCol) == F.col('maxItem'))\
            .select(self.userCol, self.itemCol)\
            .withColumnRenamed(self.itemCol, 'last_item')
        
        watched_movies_df = df\
            .groupby(self.userCol)\
            .agg(F.collect_set(self.itemCol).alias('watched'))
        
        most_popular_item = df\
            .groupBy(self.itemCol)\
            .agg(F.count(self.userCol).alias('count'))\
            .sort(F.col('count').desc())\
            .take(1)[0][self.itemCol]
        
        return AssociationRulesModel(scores_df, last_movie_df, watched_movies_df, most_popular_item,
                                     self.userCol)

In [26]:
base_ar_model = AssociationRules('userId', 'movieId','timestamp')
base_ar_model = base_ar_model.fit(train_df.filter(F.col('rating')>=3.).limit(2500))

In [27]:
base_ar_result = base_ar_model.predict(val_df).cache()

In [28]:
base_ar_result.take(2)

[Row(userId=496, recommendations=[1, 1, 260, 260, 356, 356, 440, 440, 910, 910, 1196, 1196, 1210, 1210, 1234, 1234, 1270, 1270, 1304, 1304, 1485, 1485, 2067, 2067, 5060, 5060]),
 Row(userId=833, recommendations=[2571, 2571, 2571, 2571, 2571, 2571, 2571, 2571, 48516, 48516, 48516, 48516, 48516, 48516, 48516, 48516])]

In [ ]:
base_ar_result.count()

In [ ]:
val_df.select('userId').distinct().count()

In [ ]:
#Проверка, что не рекомендует фильмы из списка запрещенных. Все метрики должны быть по нулям.
base_ar_metrics = evaluate_movies(base_ar_result, train_df)
base_ar_metrics

In [ ]:
base_ar_metrics = evaluate_movies(base_ar_result, val_df)

In [ ]:
all_metrics = {}
all_metrics['base AR'] = base_ar_metrics

## Evaluation Results

Сравните реализованные методы с помощью выбранных метрик. Не забывайте про оптимизацию гиперпараметров.

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################